# `mumble` creates input transformers for making documents.

![](https://media.giphy.com/media/3oz8xAcfZkJ0kkntoQ/giphy.gif)

In [1]:
    from toolz.curried import *
    from textwrap import indent
    from textwrap import dedent
    from inspect import getfullargspec
    choices = list()
    
    show = True

# `mumble` is fucking weird

Use type annotations to define a criteria for a transformer.  Place the return source code in the docstring.  Told you!

In [2]:
    @choices.append
    def yaml(str: lambda x: x.strip().startswith('---')):
        """locals().update(dict(
            sum((
                list(dict.items()) for dict in __import__('yaml').safe_load_all('''{}''')
                ), [])))"""

In [3]:
    @choices.append
    def graphviz(str: lambda x: x.strip().startswith('graph') or x.strip().startswith('digraph')):
        """__import__('IPython').display.display(
            __import__('graphviz').Source('''{}'''))"""

In [4]:
    @choices.append
    def iframe(str: lambda x: x.strip().startswith('http')):
        """__import__('IPython').display.display(
            __import__('IPython').display.IFrame(
            '''{}''', 800, 600))"""

In [5]:
    from nbconvert.filters.markdown_mistune import IPythonRenderer, MarkdownWithMath
    class Markdown(MarkdownWithMath):
        """A mistune.Markdown object that accumulates the source code in the markdown body.
        """
        def render(Markdown, text):
            from nbconvert.filters import ipython2python
            Markdown.renderer.source = """"""
            return [super().render(text), ipython2python(Markdown.renderer.source)][-1]

    class Renderer(IPythonRenderer):
        """A mistune.Renderer to use with `aye.Markdown`."""
        def __init__(Renderer, *args, **kwargs): 
            Renderer.source = super().__init__(*args, **kwargs) or """"""

        def block_code(Renderer, str, lang=None):
            Renderer.source += '\n' + str
            return super().block_code(str, lang=lang)

In [6]:
    from IPython.core.inputtransformer import InputTransformer
    from IPython.core.inputsplitter import IPythonInputSplitter
    
    class Transformer(__import__('collections').UserList, InputTransformer):
        push = __import__('collections').UserList.append
        def reset(Transformer): 
            from inspect import getfullargspec
            from IPython import display
            source, Transformer.data = '\n'.join(Transformer), []                
            if show and source and source.splitlines()[0].strip(): display.display(display.Markdown(source))
            source = Markdown(Renderer()).render(source)
            for callable in choices:
                spec = getfullargspec(callable)
                if spec.annotations[spec.args[0]](source):
                    source = callable.__doc__.format(source)
                    break
            return _NATIVE_TRANSFORM.transform_cell(source)
        
        def register(Transformer, object):
            TRANSFORMERS += object,

In [7]:
    def load_ipython_extension(ip=None):
        from IPython import get_ipython
        global _NATIVE_TRANSFORM
        ip = ip or get_ipython()
        _NATIVE_TRANSFORM = globals().get('_NATIVE_TRANSFORM', ip.input_transformer_manager)
        ip.input_transformer_manager = IPythonInputSplitter([], [], [Transformer()])
    
    def unload_ipython_extension(ip=None):
        from IPython import get_ipython
        global _NATIVE_TRANSFORM
        ip = ip or get_ipython()
        ip.input_transformer_manager = globals().get('_NATIVE_TRANSFORM', ip.input_transformer_manager)

    load_ipython_extension()

# This is yaml
    
    ---
    a: 10

    assert 'a' in locals()

A literate graph builder 
    
    graph graphname {
         a[href="http://google.com" target="_blank" label="Sugar"] ;
        a -- b -- c;
         b -- d;

    }

# 